In [ ]:
 """
2601547: 

Note, my comments have been labelled as 2601547 as there were some comments in code from the practicals and I wanted to keep them as 
they are helpful in explaining. Anything not labelled explicitly 2601547 is a comment from the previous author of that lab code

"""

##################################################################################################################################################################################################################
#                                                                                                IMPORT LIBRARIES                                                                                                #
##################################################################################################################################################################################################################

import numpy as np
import matplotlib.pyplot as plt

##################################################################################################################################################################################################################
#                                                                                                INPUT PARAMETERS                                                                                                #
##################################################################################################################################################################################################################

"""I will come back to this as what"""

parameters = {
    'alpha': 1.0,
    'beta': 0.01,
    'delta': 0.25,
    'gamma': 1.0,
    'K': 1000000000000000000000000000000000000000000000000000000.0, # its high as I just wanted to include the code at this point rather than just think about parameters
    'current_velocity': 1.0,
    'diffusion': 0.01,
    'PERT_P': 200, # Peturb to Prey
    'PERT_Z': 0, # Peturb to Predator
    # Grid stuff:
    'nx': 5,
    'nz':5,
    # Simulation settings:
    'timesteps': 10000,
    'domain_size': (10, 10) # physical size of area which effects flux calcs
}

##################################################################################################################################################################################################################
#                                                                                             TIMESTEP DETERMINATION                                                                                             #
##################################################################################################################################################################################################################

def stable_time_step(vx, Dx, Dz, dx, dz):
    """
    2601547: 
    
    This is code / function taken directly from "pollution.py" (GEOGM0058 Practical 7) with one ammendment made i.e. 
    return dt -> return min(dt, 0.002), which essentially just outputs 0.02, which adds some stability for the timesteps. 
    This value is taken from the 'GEOGM0058 Practical 4: Lotka-Volterra Predator Prey' file, specifically 4.3 with the "lv_pp_model" 
    model which I used to initally calibrate this model. Otherwise the functions show a lot of instability as the calculted dt term 
    is too large to capture the predator-prey dynamics, rather than just the advection-diffusion dynamics
    
    """
    dx2, dz2 = dx*dx, dz*dz
    dt_diff = dx2 * dz2 / (2 * (np.max(Dx)+np.max(Dz)) * (dx2 + dz2 ) + 1.0e-10) # stable time step. 
    dt_cfl = dx / (np.max(vx) + 1.0e-10) # advection stable step
    dt = min(dt_cfl, dt_diff)*0.5

    return min(dt, 0.02)

##################################################################################################################################################################################################################
#                                                                                           ADVECTION / DIFFUSION MODEL                                                                                          #
##################################################################################################################################################################################################################

def euler_advection_diffusion_timestep(c0, vx, Dx, Dz, src, dt, dx, dz, upwind=True):
    """ 
    2601547: 
    
    This is code / function again taken from "pollution.py" (GEOGM0058 Practical 7), with some ammedments made around the way c is 
    updated acoss the grid. The model this function is taken from was designed to simulate pollutants in the air, looking at almost 
    a cross-sectional view of the earth as a grid i.e. ground as the bottom row cells and the atmosphere as the top. My approach 
    leverages the code, but instead I immagine looking top-down at a body of water as if e.g. from a satellite. In the former approach, 
    c values, which represent the concentration of pollution in a cell and hence determine the diffusion coefficients (these are 
    calculated by comparison of concentrations in neighbouring cells, more information in the lab file 'GEOGM0058 Practical 7: 
    Atmospheric pollution dispersal as an example of advection and diffusion processes') are updated depending on their position too. 
    For example, the bottom row of cells which represented the "ground boundary" would replicate c values in the row above, so that
    concentrations would be the same and hence the flux / diffusion coefficient between the two would be 0. This makes sense in that 
    context, as it prevents depoisiton (as intended). Similarly, at the top layer which represented the atmosphere, depending on a term 
    called "inversion", the values of c in the top row could be set to 0 (inversion=FALSE) essentially acting as a sink, or could
    use a similar approach to the ground boundary. In this regard, only the central cells actually had c / concentration values which
    updated depending on diffusion etc.

    My changes to the code therefore are limited - I keep all the diffusive elements, but remove any reference to inversion as this
    dynamic is not relevant for a top-down view of a body of water. I update the code so that all grid cells update their c values
    dependant on diffusive / advection dynamics only, and no values are set to 0 which would create an artificial sink etc. I finally
    add a condition that c cannot be negative, as this prevents any non-sensical cases


    ++++++++++++++++++++++++++ Comments from creator below ++++++++++++++++++++++++++

    
    Evolve the advection diffusion equation for pollution concentration 
    by one time step. Spatial discretization using the finite volume method
    on a rectangular mesh of rectangular cells. Forward Euler timestep
     
    Advection is limited to wind parallel to the x-axis
    
    Note the time-step and advection schemes here are for illustration. Real
    codes use better schemes (usually :)
    
    parameters
    ----------
    c0 ((nz, nx) ndarray) : cell-centered concentration at the start of the time step
    vx ((nz, nx+1) ndarray) : horizontal wind velocity at x-faces
    Dx ((nz, nx+1) ndarray) : horizontal diffusion coeffcient at x-faces
    Dz ((nz+1, nx) ndarray) : horizontal diffusion coeffcient at z-faces
    src: concentration source per unit time
    dt (float) : timestep. not checked for stability
    
    inversion: affects the boundary condition at the top boundary: 
               when true, pollution cannot cross the inversion.
               when false, concentration set to 0 at the top
    
    upwind: choice of advection schemne. When true, first-order upwind.
    when false, central difference
    
    returns
    ------
    c (ndarray): updated concentration
    
    """
    nz, nx = c0.shape
    c = c0.copy()
    
    #storage for face-centered fluxes. index j correspomnd to 'w' face of cell j
    Fx = np.zeros((nz, nx + 1))
    Fz = np.zeros((nz +1 , nx))
    
    #Diffusive fluxes (per unit volume) 
    Fx[:, 1:nx] = - Dx[:, 1:nx] * (c[:,1:nx] - c[:,0:nx-1]) / (dx*dx)
    Fz[1:nz, :] = - Dz[1:nz, :] * (c[1:nz,:] - c[0:nz-1,:]) / (dz*dz)
    
    #Advective fluxes
    if upwind:
       Fx[:, 1:nx] += np.where(vx[:,1:nx] > 0, vx[:,1:nx] * c[:,0:nx-1], -vx[:,1:nx]*c[:,1:nx]) / dx     
    else:
       Fx[:, 1:nx] += vx[:,1:nx] * (c[:,0:nx-1] + c[:,1:nx]) * 0.5 / dx
    
    # 2601547: Here I have made changes to the way c updates the array, by updating the grid cells which are updated from internal to all                 
    c[0:nz, 0:nx] += dt * (
        -(Fx[0:nz, 1:nx+1] - Fx[0:nz, 0:nx])
        -(Fz[1:nz+1, 0:nx] - Fz[0:nz, 0:nx])
        + src[0:nz, 0:nx]
    )

    # 2601547: I removed all boundary conditions as they are not 
    c = np.maximum(c, 0) # 2601547: I prevent negative values of c
    
    return c

##################################################################################################################################################################################################################
#                                                                                         DOMAIN / GRID SIZE DETERMINATION                                                                                       #
##################################################################################################################################################################################################################

def create_grid(P0, Z0, domain_size, parameters):
    """
    2601547: 
    
    This is code extracted from the the 'transport_pollution' function in "pollution.py" (GEOGM0058 Practical 7). I do not use 
    the funtion, however to to create a grid to run the simulations, I needed this section. The same grid used for advection-
    diffusion / atmospheric pollution is used, with the size dependant on the initial conditions for prey, and with a check that
    is the same size of the predator's 

    I have written out the paramters in the same style as the example code:

    parameters
    ----------
    P0 (ndarray) : initial prey density array
    Z0 (ndarray) : initial predator density array
    domain_size (tuple) : physical size of the spatial domain (x, z)
    vx ((nz, nx+1) ndarray) : horizontal current velocity at x-faces
    Dx ((nz, nx+1) ndarray) : horizontal diffusion coeffcient at x-faces
    Dz ((nz+1, nx) ndarray) : horizontal diffusion coeffcient at z-faces
    dt (float) : timestep
    
    """
    current_velocity, diffusion = parameters['current_velocity'], parameters['diffusion']
    
    # 2601547: Here I added a check that both P0 and Z0 are the same shape array. This is a safeguard that was requested by Dr Cornfold when discussing this code
    if P0.shape != Z0.shape:
        raise ValueError(f"P0 and Z0 must have the same shape. Got P0.shape={P0.shape}, Z0.shape={Z0.shape}")

    # 2601547: The grid is constructed here. I use a similar approach to the ap/d lab (see pollution.py, line 47) i.e. the grid size is determined by the initial conditions for prey
    nz, nx = P0.shape 
    # 2601547: Cell dimensions is also determined by my spatial domain size (see pollution.py, line 123)
    dx = domain_size[0] / nx 
    dz = domain_size[1] / nz

    # 2601547: Current speed / advection at cell x - faces (see pollution.py, line 137)
    vx = np.zeros((nz, nx + 1)) + current_velocity
    # 2601547: Diffusion at cell x-faces (see pollution.py, line 139)
    Dx = np.full((nz, nx + 1), diffusion)
    # 2601547: Diffusion at cell z-faces (see pollution.py, line 141)
    Dz = np.full((nz + 1, nx), diffusion)

    dt = stable_time_step(vx, Dx, Dz, dx, dz)
    print(f"Timestep: {dt:.5f}s") # 2601547: Minor ammendment to print statement

    return {'nz': nz, 'nx': nx, 'dx': dx, 'dz': dz, 'vx': vx, 'Dx': Dx, 'Dz': Dz, 'dt': dt, 'P': P0.copy(), 'Z': Z0.copy()}

##################################################################################################################################################################################################################
#                                                                                              LOTKA-VOLTERRA MODEL                                                                                              #
##################################################################################################################################################################################################################

def lotka_volterra_sources(P, Z, parameters):
    """
    2601547: 

    Based on GEOGM0058 Practical 7 part 4.3 and 4.4, functions 'lv_pp_model' and 'alt_pp_model' respectively, I have laid out
    the Lotka-Volterra Model I described in the paper i.e classical competitive with carry capacity. I used greek symbols in
    notation instead of "ABCD", as I thought it was more relevant in an academic writing context, so a parameter disctionary was
    used
    
    I have calculated the indvidual components that contribute to both prey and predator growth / death. In the advection-
    diffusion model, there is a source / sink "S" component. This is caculated as the growth - death i.e. a source if growth 
    exceeds death and a sink if the opposite is true. The src terms are calculated and returned so that they can be passed 
    through the advection-diffusion model in the next step of the simulation

    """
    alpha, beta, delta, gamma, K = parameters['alpha'], parameters['beta'], parameters['delta'], parameters['gamma'], parameters['K']

    prey_growth = alpha * P * (1.0 - P / K)
    prey_death = -beta * P * Z
    predator_growth = delta * beta * P * Z
    predator_death = -gamma * Z

    src_prey = prey_growth + prey_death
    src_pred = predator_growth + predator_death

    return src_prey, src_pred

##################################################################################################################################################################################################################
#                                                                                            ADVECTION-DIFFUSION MODEL                                                                                           #
##################################################################################################################################################################################################################

def predator_prey_advection_diffusion_step(P, Z, vx, Dx, Dz, parameters, dt, dx, dz):
    """
    2601547: 

    Here I pass through for each Prey and Predators values. As per the 'euler_advection_diffusion_timestep' function, I pass through 
    parameters where "P or "Z" are input as concentrtion i.e. "c0" and "src_prey" and "src_pred" as "src". In my current simulations,
    upwind is not used, so I have left it out

    This function is the crux of how the process works. Each simulation is seperated into two distinct steps which effect P and Z values
    for the next step:
    
    1) P and Z density / concentraion values are calculated in each iteration per cell based purely of the Lotka Volterra model i.e. 
    predation / competitve dynamics
    
    2) Then diffusion / advection is applied to that via the 'euler_advection_diffusion_timestep' function, resulting in P_updated and 
    Z_updated density values i.e. a new concentraion in each cell, caused due by spatial dynamics rather than because of predation / births
    / deaths, of spatial dynamics

    The new P_updated and Z_updated values are then used as P and Z respectuvely in the next iteration ensuring that both dynamics are
    captured

    """
    src_prey, src_pred = lotka_volterra_sources(P, Z, parameters)

    P_updated = euler_advection_diffusion_timestep(P, vx, Dx, Dz, src_prey, dt, dx, dz)
    Z_updated = euler_advection_diffusion_timestep(Z, vx, Dx, Dz, src_pred, dt, dx, dz)

    return P_updated, Z_updated

##################################################################################################################################################################################################################
#                                                                                            ADVECTION-DIFFUSION MODEL                                                                                           #
##################################################################################################################################################################################################################

def run_simulation_v1():
    """
    2601547: 

    I fill out arrays P0 and Z0 i.e. initial conditions for P and Z densities with the equilibria conditions assuming spatial homogenuity 
    i.e. no advection / diffusion (plus a pertubing variable). These are akin to the starting conditions to the Lotka-Volterra model 
    described in 4.3, (specifically, "x_y_initial = (D/(C*B) + PERT_X , A/B)") except, I have a carrying capacity, so the solutions are 
    slightly different. This ensures, regardless of the starting coefficient / greek parameter values, the model will be in equilibrium, 
    if there is no advection or diffusion (i.e. they both equal 0). However, this is not true if there is any spatial elements, which is 
    the innovative aspect of this model, so it is somewhat trivial. It is, however, a good starting point, for identifying closed orbits, 
    even with spatial dynamics

    The first plot is, again, the very basic one from 4.3, which was helpful to begin thinking about presentation of results

    """
    global grid # 2601547: This allows me to use the grid value in other (i.e. the next plot).
    alpha, beta, delta, gamma, K, nx, nz = parameters['alpha'], parameters['beta'], parameters['delta'], parameters['gamma'], parameters['K'], parameters['nx'], parameters['nz']
    PERT_P, PERT_Z, timesteps, domain_size = parameters['PERT_P'], parameters['PERT_Z'], parameters['timesteps'], parameters['domain_size'] # 2601547: seperate line purely for my own readability
    P0 = np.full((nz, nx), (gamma / (delta * beta)) + PERT_P)  # 2601547: equilibrium condition + a pertubance
    Z0 = np.full((nz, nx), ((alpha / beta) * (1 - (gamma / (delta * beta * K)))) + PERT_Z ) # 2601547: equilibrium condition + a pertubance
    
    # 2601547: This allows me to see the actual equilbrium values depending on the input parameters. In this case, I just wanted to check they were the same as the lab
    eq_p = gamma / (delta * beta)
    eq_z = (alpha / beta) * (1 - (gamma / (delta * beta * K)))
    print(f"Equiilibrium Prey Value = {eq_p:.0f}")
    print(f"Equiilibrium Prey Value = {eq_z:.0f}")
    
    # 2601547: This calls / runs the grid function for the simulation
    grid = create_grid(P0, Z0, domain_size, parameters)
    P = grid['P']
    Z = grid['Z']

    # 2601547: This stores the initial conditions
    P_history = np.zeros((timesteps, nz, nx))
    Z_history = np.zeros((timesteps, nz, nx))
    P_history[0, :, :] = P
    Z_history[0, :, :] = Z

    # 2601547: Starting from t=1, I use a For loop to run the simulation through the timesteps value. P_history and Z_history track the values over time, hence the 3D
    for t in range(1, timesteps):
        P, Z = predator_prey_advection_diffusion_step(
            P, Z, grid['vx'], grid['Dx'], grid['Dz'], parameters, grid['dt'], grid['dx'], grid['dz']
        )
        P_history[t, :, :] = P
        Z_history[t, :, :] = Z

        # 2601547: I used this function to get some data on what the evolution and variance of the populations look like at each timestep. It was helpful for debugging some issues too
        if t % (timesteps / 10) == 0:
            print(f"Timestep {t}: P min={np.min(P):.4f}, P max={np.max(P):.4f}, Z min={np.min(Z):.4f}, Z max={np.max(Z):.4f}")

    # Phase Plot code taken and ammended from 4.3 in GEOGM0058 Practical 4
    fig, axes = plt.subplots(nz, nx, figsize=(12, 12), sharex=True, sharey=True)
    for i in range(nz):
        for j in range(nx):
            ax = axes[i, j]
            ax.plot(P_history[:, i, j], Z_history[:, i, j], label=f"Cell ({i}, {j})")
            ax.set_title(f"Phase Plot: Cell ({i}, {j})")
            ax.set_xlabel("Prey Density")
            ax.set_ylabel("Predator Density")
 #           ax.axhline(alpha / beta, color='k',ls='--',label=r'$y = a/b$', lw=0.005)
 #           ax.axvline(delta /(gamma * beta), color='k',ls=':',label=r'$x = d/bc$', lw=0.005)
            ax.legend()
    plt.tight_layout()
    plt.show()

##################################################################################################################################################################################################################
#                                                                                                 RUN SIMULATION                                                                                                 #
##################################################################################################################################################################################################################

run_simulation_v1()
